# [ADVANCED] Udaplay Project

## Part 03 - Advanced Agent with Long-Term Memory & Explicit State Machine

This notebook demonstrates advanced agent features:

### 🎯 Key Features

1. **Long-Term Memory**: Persistent storage of game facts across sessions using vector embeddings
   - Facts learned from queries are stored permanently
   - Memory is searched semantically before other sources
   - Reduces redundant searches and API costs

2. **Explicit State Machine**: Pre-defined tool nodes instead of dynamic tool executor
   - Each tool is a dedicated step/node in the workflow
   - Clear, deterministic execution path
   - Easy to visualize and debug
   - Conditional branching based on evaluation results

### 🔄 Workflow
```
entry → retrieve_memory → retrieve_game_db → evaluate
                                                ↓
                              useful? YES → generate_answer
                                      NO  → web_search → generate_answer
                                                            ↓
                                                       store_memory → end
```

### Setup

In [32]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [33]:
import os
import json
from typing import TypedDict, Optional, Dict, List
from datetime import datetime

import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv
from tavily import TavilyClient
from pydantic import BaseModel, Field

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool
from lib.parsers import PydanticOutputParser
from lib.state_machine import StateMachine, Step, EntryPoint, Termination, Run
from lib.memory import ShortTermMemory, LongTermMemory, MemoryFragment
from lib.vector_db import VectorStoreManager
from lib.documents import Document

In [34]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

print("✓ Environment variables loaded")
print(f"✓ OPENAI_API_KEY: {'Found' if OPENAI_API_KEY else 'NOT FOUND'}")
print(f"✓ TAVILY_API_KEY: {'Found' if TAVILY_API_KEY else 'NOT FOUND'}")

✓ Environment variables loaded
✓ OPENAI_API_KEY: Found
✓ TAVILY_API_KEY: Found


### Long-Term Memory Setup

Initialize persistent memory for storing game facts across sessions.

In [35]:
# Create VectorStoreManager for long-term memory
# Note: VectorStoreManager uses in-memory storage by default
# For persistent storage across sessions, you would need to modify VectorStoreManager
# or use ChromaDB PersistentClient directly

memory_manager = VectorStoreManager(
    openai_api_key=OPENAI_API_KEY  # Corrected parameter name
)

# Initialize long-term memory
long_term_memory = LongTermMemory(db=memory_manager)

print("✓ Long-term memory initialized")
print("✓ Note: Memory is in-memory only (will reset when kernel restarts)")
print("✓ For persistent memory, use ChromaDB PersistentClient directly")

✓ Long-term memory initialized
✓ Note: Memory is in-memory only (will reset when kernel restarts)
✓ For persistent memory, use ChromaDB PersistentClient directly


### Initialize Game Database & Tools

Load the game vector database and define all tools.

In [36]:
# Initialize ChromaDB client for game database
chroma_client = chromadb.PersistentClient(path="chromadb")
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY,
    model_name="text-embedding-3-small"
)
game_collection = chroma_client.get_collection("udaplay_games", embedding_function=embedding_fn)

# Initialize Tavily client for web search
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

print("✓ Game database loaded")
print(f"✓ Collection: udaplay_games")
print(f"✓ Web search client ready")

✓ Game database loaded
✓ Collection: udaplay_games
✓ Web search client ready


### Memory Tools

Tools for storing and retrieving facts from long-term memory.

In [37]:
@tool
def store_game_fact(fact: str, category: str = "general") -> str:
    """
    Store an important game fact in long-term memory for future retrieval.
    
    Args:
        fact: The game fact to store (e.g., "Super Mario 64 was released in 1996")
        category: Category of the fact (e.g., "release_date", "platform", "general")
    
    Returns:
        Confirmation message
    """
    memory_fragment = MemoryFragment(
        content=fact,
        owner="udaplay_agent",
        namespace=category
    )
    
    long_term_memory.register(memory_fragment)
    
    return json.dumps({
        "status": "stored",
        "fact": fact,
        "category": category
    })

@tool
def retrieve_memory(query: str, category: str = "general") -> str:
    """
    Search long-term memory for relevant game facts.
    
    Args:
        query: The search query
        category: Category namespace to search in
    
    Returns:
        Retrieved facts from memory
    """
    results = long_term_memory.search(
        query_text=query,
        owner="udaplay_agent",
        limit=3,
        namespace=category
    )
    
    facts = []
    for fragment in results.fragments:
        facts.append({
            "fact": fragment.content,
            "timestamp": datetime.fromtimestamp(fragment.timestamp).isoformat(),
            "category": fragment.namespace
        })
    
    return json.dumps(facts, indent=2)

print("✓ Memory tools defined: store_game_fact, retrieve_memory")

✓ Memory tools defined: store_game_fact, retrieve_memory


### Game Research Tools

Original tools for vector DB search, evaluation, and web search.

In [38]:
@tool
def retrieve_game(query: str) -> str:
    """
    Semantic search: Finds most similar results in the vector DB
    
    Args:
        query: a question about game industry
    
    Returns:
        Results as a list with game information
    """
    results = game_collection.query(
        query_texts=[query],
        n_results=5
    )
    
    formatted_results = []
    if results['metadatas'] and results['metadatas'][0]:
        for metadata in results['metadatas'][0]:
            formatted_results.append({
                "Platform": metadata.get("Platform", "Unknown"),
                "Name": metadata.get("Name", "Unknown"),
                "YearOfRelease": metadata.get("YearOfRelease", "Unknown"),
                "Description": metadata.get("Description", "No description available")
            })
    
    return json.dumps(formatted_results, indent=2)

class EvaluationReport(BaseModel):
    """Evaluation of retrieved documents"""
    useful: bool = Field(description="Whether the documents are useful to answer the question")
    description: str = Field(description="Detailed explanation about the evaluation result")

@tool
def evaluate_retrieval(question: str, retrieved_docs: str) -> str:
    """
    Evaluates if retrieved documents are sufficient to answer the question.
    
    Args:
        question: original question from user
        retrieved_docs: retrieved documents
    
    Returns:
        Evaluation with useful flag and description
    """
    llm_judge = LLM(model="gpt-4o-mini", temperature=0.0, api_key=OPENAI_API_KEY)
    
    evaluation_prompt = f"""Your task is to evaluate if the documents are enough to respond the query.
Give a detailed explanation, so it's possible to take an action to accept it or not.

User Question: {question}

Retrieved Documents:
{retrieved_docs}

Evaluate whether these documents contain sufficient information to answer the user's question."""
    
    response = llm_judge.invoke(
        input=evaluation_prompt,
        response_format=EvaluationReport
    )
    
    parser = PydanticOutputParser(model_class=EvaluationReport)
    evaluation = parser.parse(response)
    
    return json.dumps({
        "useful": evaluation.useful,
        "description": evaluation.description
    }, indent=2)

@tool
def game_web_search(question: str) -> str:
    """
    Web search: Searches the web for information about games
    
    Args:
        question: a question about game industry
    
    Returns:
        Search results from the web
    """
    response = tavily_client.search(
        query=question,
        max_results=5
    )
    
    results = []
    for result in response.get('results', []):
        results.append({
            "title": result.get("title", ""),
            "url": result.get("url", ""),
            "content": result.get("content", "")
        })
    
    return json.dumps(results, indent=2)

print("✓ Game tools defined: retrieve_game, evaluate_retrieval, game_web_search")

✓ Game tools defined: retrieve_game, evaluate_retrieval, game_web_search


### Advanced Agent State Schema

Enhanced state that tracks all intermediate results through the workflow.

In [39]:
class AdvancedAgentState(TypedDict):
    """State schema for the advanced agent with explicit tracking"""
    
    # Input
    user_query: str
    session_id: str
    owner: str  # For memory isolation
    
    # Intermediate results from each step
    memory_results: Optional[str]  # Results from long-term memory
    retrieved_docs: Optional[str]  # Results from vector DB
    evaluation_result: Optional[Dict]  # {useful: bool, description: str}
    web_results: Optional[str]  # Results from web search
    final_answer: Optional[str]  # Generated answer
    
    # Metadata
    messages: List[dict]
    total_tokens: int

print("✓ AdvancedAgentState schema defined")

✓ AdvancedAgentState schema defined


### Step Functions - Pre-defined Tool Nodes

Each tool becomes a dedicated step/node in the state machine.

In [40]:
def retrieve_from_memory_step(state: AdvancedAgentState) -> Dict:
    """Step 1: Search long-term memory for relevant facts"""
    print("  📚 Searching long-term memory...")
    
    memory_results = retrieve_memory(state["user_query"], "general")
    
    return {
        "memory_results": memory_results
    }

def retrieve_game_step(state: AdvancedAgentState) -> Dict:
    """Step 2: Search vector database for game information"""
    print("  🎮 Searching game database...")
    
    retrieved_docs = retrieve_game(state["user_query"])
    
    return {
        "retrieved_docs": retrieved_docs
    }

def evaluate_retrieval_step(state: AdvancedAgentState) -> Dict:
    """Step 3: Evaluate if retrieved documents are sufficient"""
    print("  🔍 Evaluating retrieval quality...")
    
    # Combine memory and vector DB results
    all_docs = state.get("retrieved_docs", "")
    memory = state.get("memory_results", "")
    
    if memory and memory != "[]":
        all_docs = f"Memory:\n{memory}\n\nVector DB:\n{all_docs}"
    
    evaluation_json = evaluate_retrieval(state["user_query"], all_docs)
    evaluation = json.loads(evaluation_json)
    
    return {
        "evaluation_result": evaluation
    }

def web_search_step(state: AdvancedAgentState) -> Dict:
    """Step 4: Fallback web search when internal sources are insufficient"""
    print("  🌐 Searching the web...")
    
    web_results = game_web_search(state["user_query"])
    
    return {
        "web_results": web_results
    }

def generate_answer_step(state: AdvancedAgentState) -> Dict:
    """Step 5: Generate final answer using all available information"""
    print("  ✍️  Generating answer...")
    
    # Build context from all sources
    context_parts = []
    
    if state.get("memory_results") and state["memory_results"] != "[]":
        context_parts.append(f"Long-term Memory:\n{state['memory_results']}")
    
    if state.get("retrieved_docs"):
        context_parts.append(f"Game Database:\n{state['retrieved_docs']}")
    
    if state.get("web_results"):
        context_parts.append(f"Web Search Results:\n{state['web_results']}")
    
    context = "\n\n".join(context_parts)
    
    # Generate answer
    llm = LLM(model="gpt-4o-mini", temperature=0.7, api_key=OPENAI_API_KEY)
    
    prompt = f"""You are UdaPlay, an AI Research Agent for video games.

User Question: {state['user_query']}

Available Information:
{context}

Provide a clear, accurate answer to the user's question. Be concise but informative.
If you use information from web search, mention it."""
    
    response = llm.invoke(prompt)
    
    return {
        "final_answer": response.content,
        "total_tokens": state.get("total_tokens", 0) + (response.token_usage.total_tokens if response.token_usage else 0)
    }

def store_memory_step(state: AdvancedAgentState) -> Dict:
    """Step 6: Extract and store important facts in long-term memory"""
    print("  💾 Storing facts in memory...")
    
    # Use LLM to extract facts worth storing
    llm = LLM(model="gpt-4o-mini", temperature=0.0, api_key=OPENAI_API_KEY)
    
    extraction_prompt = f"""Extract important game facts from this conversation that should be stored for future reference.

Question: {state['user_query']}
Answer: {state.get('final_answer', '')}

Return a JSON list of facts to store. Each fact should be a clear, standalone statement.
Only include facts that are verifiable and useful.

Format: {{"facts": ["fact1", "fact2", ...]}}

If no facts worth storing, return {{"facts": []}}"""
    
    response = llm.invoke(extraction_prompt)
    
    try:
        facts_data = json.loads(response.content)
        facts = facts_data.get("facts", [])
        
        for fact in facts:
            store_game_fact(fact, "general")
            print(f"    ✓ Stored: {fact[:80]}...")
    except:
        print("    ℹ No facts extracted for storage")
    
    return {}

print("✓ All step functions defined")

✓ All step functions defined


### Build Explicit State Machine

Create the workflow with pre-defined tool nodes and conditional branching.

In [41]:
def build_advanced_workflow() -> StateMachine[AdvancedAgentState]:
    """Build state machine with explicit tool nodes"""
    
    machine = StateMachine[AdvancedAgentState](AdvancedAgentState)
    
    # Create all nodes/steps
    entry = EntryPoint[AdvancedAgentState]()
    memory_retrieval = Step[AdvancedAgentState]("memory_retrieval", retrieve_from_memory_step)
    game_retrieval = Step[AdvancedAgentState]("game_retrieval", retrieve_game_step)
    evaluation = Step[AdvancedAgentState]("evaluation", evaluate_retrieval_step)
    web_search = Step[AdvancedAgentState]("web_search", web_search_step)
    generate_answer = Step[AdvancedAgentState]("generate_answer", generate_answer_step)
    store_memory = Step[AdvancedAgentState]("store_memory", store_memory_step)
    termination = Termination[AdvancedAgentState]()
    
    # Add all steps to machine
    machine.add_steps([
        entry, memory_retrieval, game_retrieval, evaluation,
        web_search, generate_answer, store_memory, termination
    ])
    
    # Connect steps in workflow
    machine.connect(entry, memory_retrieval)
    machine.connect(memory_retrieval, game_retrieval)
    machine.connect(game_retrieval, evaluation)
    
    # Conditional branching after evaluation
    def check_evaluation(state: AdvancedAgentState):
        """Decide: if useful → answer, else → web_search"""
        eval_result = state.get("evaluation_result", {})
        if eval_result.get("useful"):
            print("  ✅ Evaluation: Documents are sufficient")
            return generate_answer
        else:
            print("  ⚠️  Evaluation: Documents insufficient, need web search")
            return web_search
    
    machine.connect(evaluation, [generate_answer, web_search], check_evaluation)
    machine.connect(web_search, generate_answer)
    machine.connect(generate_answer, store_memory)
    machine.connect(store_memory, termination)
    
    return machine

print("✓ Workflow builder function defined")

✓ Workflow builder function defined


### Advanced Agent Class

Agent that uses the explicit state machine and long-term memory.

In [42]:
class AdvancedAgent:
    """Advanced agent with long-term memory and explicit state machine"""
    
    def __init__(self, model_name: str = "gpt-4o-mini", temperature: float = 0.7):
        self.model_name = model_name
        self.temperature = temperature
        self.workflow = build_advanced_workflow()
        self.short_term_memory = ShortTermMemory()
    
    def invoke(self, query: str, session_id: str = "default", owner: str = "default_user") -> Run:
        """Execute the agent workflow on a query"""
        
        # Create session if it doesn't exist
        self.short_term_memory.create_session(session_id)
        
        # Create initial state
        initial_state: AdvancedAgentState = {
            "user_query": query,
            "session_id": session_id,
            "owner": owner,
            "memory_results": None,
            "retrieved_docs": None,
            "evaluation_result": None,
            "web_results": None,
            "final_answer": None,
            "messages": [],
            "total_tokens": 0
        }
        
        # Run workflow
        run = self.workflow.run(initial_state)
        
        # Store in short-term memory
        self.short_term_memory.add(run, session_id)
        
        return run
    
    def get_session_runs(self, session_id: str = "default") -> List[Run]:
        """Get all runs for a session"""
        return self.short_term_memory.get_all_objects(session_id)

# Create the agent
advanced_agent = AdvancedAgent(model_name="gpt-4o-mini", temperature=0.7)

print("✓ AdvancedAgent created successfully")
print(f"✓ Model: gpt-4o-mini")
print(f"✓ Workflow: Explicit state machine with 6 tool nodes")

✓ AdvancedAgent created successfully
✓ Model: gpt-4o-mini
✓ Workflow: Explicit state machine with 6 tool nodes


### Testing & Demonstration

Test the advanced agent with different scenarios.

#### Scenario 1: First Query (No Memory)

Expected flow: memory (empty) → vector_db → evaluate (useful) → answer → store

In [43]:
print("="*80)
print("SCENARIO 1: First-time query about Super Mario 64")
print("="*80)

run1 = advanced_agent.invoke(
    query="When was Super Mario 64 released and on what platform?",
    session_id="demo_1",
    owner="user1"
)

final_state = run1.get_final_state()

print("\n" + "─"*80)
print("FINAL ANSWER:")
print("─"*80)
print(final_state.get("final_answer", "No answer generated"))
print(f"\nTokens used: {final_state.get('total_tokens', 0)}")

SCENARIO 1: First-time query about Super Mario 64
[StateMachine] Starting: __entry__
  📚 Searching long-term memory...
[StateMachine] Executing step: memory_retrieval
  🎮 Searching game database...
[StateMachine] Executing step: game_retrieval
  🔍 Evaluating retrieval quality...
[StateMachine] Executing step: evaluation
  ✅ Evaluation: Documents are sufficient
  ✍️  Generating answer...
[StateMachine] Executing step: generate_answer
  💾 Storing facts in memory...
    ✓ Stored: Super Mario 64 was released in 1996....
    ✓ Stored: Super Mario 64 was released on the Nintendo 64 platform....
    ✓ Stored: Super Mario 64 is a 3D platformer....
    ✓ Stored: In Super Mario 64, Mario embarks on a quest to rescue Princess Peach....
[StateMachine] Executing step: store_memory
[StateMachine] Terminating: __termination__

────────────────────────────────────────────────────────────────────────────────
FINAL ANSWER:
────────────────────────────────────────────────────────────────────────────────


#### Scenario 2: Repeat Query (Uses Memory)

Expected flow: memory (found!) → vector_db → evaluate → answer

In [44]:
print("="*80)
print("SCENARIO 2: Repeat query (should use long-term memory)")
print("="*80)

run2 = advanced_agent.invoke(
    query="What platform was Super Mario 64 on?",
    session_id="demo_2",
    owner="user1"
)

final_state = run2.get_final_state()

print("\n" + "─"*80)
print("MEMORY CHECK:")
print("─"*80)
memory_data = final_state.get("memory_results", "[]")
if memory_data and memory_data != "[]":
    print("✓ Found in memory:")
    print(memory_data)
else:
    print("ℹ No memory found (this is expected if this is the first run)")

print("\n" + "─"*80)
print("FINAL ANSWER:")
print("─"*80)
print(final_state.get("final_answer", "No answer generated"))
print(f"\nTokens used: {final_state.get('total_tokens', 0)}")

SCENARIO 2: Repeat query (should use long-term memory)
[StateMachine] Starting: __entry__
  📚 Searching long-term memory...
[StateMachine] Executing step: memory_retrieval
  🎮 Searching game database...
[StateMachine] Executing step: game_retrieval
  🔍 Evaluating retrieval quality...
[StateMachine] Executing step: evaluation
  ✅ Evaluation: Documents are sufficient
  ✍️  Generating answer...
[StateMachine] Executing step: generate_answer
  💾 Storing facts in memory...
    ✓ Stored: Super Mario 64 was released on the Nintendo 64 platform....
[StateMachine] Executing step: store_memory
[StateMachine] Terminating: __termination__

────────────────────────────────────────────────────────────────────────────────
MEMORY CHECK:
────────────────────────────────────────────────────────────────────────────────
✓ Found in memory:
[
  {
    "fact": "Super Mario 64 was released on the Nintendo 64 platform.",
    "timestamp": "2025-10-21T14:24:07",
    "category": "general"
  },
  {
    "fact": "Sup

#### Scenario 3: Query Requiring Web Search

Expected flow: memory → vector_db → evaluate (not useful) → web_search → answer → store

In [45]:
print("="*80)
print("SCENARIO 3: Query about recent/missing game (needs web search)")
print("="*80)

run3 = advanced_agent.invoke(
    query="When is Grand Theft Auto VI expected to be released?",
    session_id="demo_3",
    owner="user1"
)

final_state = run3.get_final_state()

print("\n" + "─"*80)
print("WEB SEARCH CHECK:")
print("─"*80)
if final_state.get("web_results"):
    print("✓ Web search was used")
    web_data = json.loads(final_state["web_results"])
    print(f"Found {len(web_data)} web results")
else:
    print("ℹ Web search was not used")

print("\n" + "─"*80)
print("FINAL ANSWER:")
print("─"*80)
print(final_state.get("final_answer", "No answer generated"))
print(f"\nTokens used: {final_state.get('total_tokens', 0)}")

SCENARIO 3: Query about recent/missing game (needs web search)
[StateMachine] Starting: __entry__
  📚 Searching long-term memory...
[StateMachine] Executing step: memory_retrieval
  🎮 Searching game database...
[StateMachine] Executing step: game_retrieval
  🔍 Evaluating retrieval quality...
[StateMachine] Executing step: evaluation
  ⚠️  Evaluation: Documents insufficient, need web search
  🌐 Searching the web...
[StateMachine] Executing step: web_search
  ✍️  Generating answer...
[StateMachine] Executing step: generate_answer
  💾 Storing facts in memory...
    ✓ Stored: Grand Theft Auto VI is expected to be released on May 26, 2026....
    ✓ Stored: The release date information comes from Rockstar Games' official announcement....
    ✓ Stored: The release date has been reported across various platforms....
[StateMachine] Executing step: store_memory
[StateMachine] Terminating: __termination__

────────────────────────────────────────────────────────────────────────────────
WEB SEARCH

### Memory Inspection

View what has been stored in long-term memory.

In [46]:
print("="*80)
print("LONG-TERM MEMORY CONTENTS")
print("="*80)

# Search for Mario-related facts
print("\nSearching for 'Mario' facts:")
print("─"*80)
memory_results = long_term_memory.search(
    query_text="Mario",
    owner="udaplay_agent",
    limit=5,
    namespace="general"
)

if memory_results.fragments:
    for i, fragment in enumerate(memory_results.fragments, 1):
        timestamp = datetime.fromtimestamp(fragment.timestamp)
        print(f"\n{i}. {fragment.content}")
        print(f"   Stored: {timestamp.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"   Category: {fragment.namespace}")
else:
    print("No facts found in memory yet.")

# Search for all facts
print("\n\nSearching for all game facts:")
print("─"*80)
all_results = long_term_memory.search(
    query_text="video game",
    owner="udaplay_agent",
    limit=10,
    namespace="general"
)

if all_results.fragments:
    print(f"Total facts stored: {len(all_results.fragments)}")
    for i, fragment in enumerate(all_results.fragments, 1):
        print(f"{i}. {fragment.content[:100]}...")
else:
    print("No facts found in memory yet.")

LONG-TERM MEMORY CONTENTS

Searching for 'Mario' facts:
────────────────────────────────────────────────────────────────────────────────

1. Super Mario 64 is a 3D platformer.
   Stored: 2025-10-21 14:24:07
   Category: general

2. In Super Mario 64, Mario embarks on a quest to rescue Princess Peach.
   Stored: 2025-10-21 14:24:07
   Category: general

3. Super Mario 64 was released on the Nintendo 64 platform.
   Stored: 2025-10-21 14:24:07
   Category: general

4. Super Mario 64 was released on the Nintendo 64 platform.
   Stored: 2025-10-21 14:24:13
   Category: general

5. Super Mario 64 was released in 1996.
   Stored: 2025-10-21 14:24:07
   Category: general


Searching for all game facts:
────────────────────────────────────────────────────────────────────────────────
Total facts stored: 8
1. Super Mario 64 is a 3D platformer....
2. Super Mario 64 was released on the Nintendo 64 platform....
3. Super Mario 64 was released on the Nintendo 64 platform....
4. In Super Mario 64, Mar

### Comparison: Standard vs Advanced Agent

| Feature | Standard Agent (Part 02) | Advanced Agent (Part 03) |
|---------|-------------------------|-------------------------|
| **Memory** | Short-term only (session-based, in-memory) | Short-term + Long-term (persistent vector storage) |
| **State Machine** | Generic tool executor step | Explicit pre-defined tool nodes |
| **Workflow** | Dynamic tool selection by LLM | Deterministic flow with conditional branching |
| **Debugging** | Tools called in generic step | Each tool is a visible node |
| **Knowledge Persistence** | Lost after session ends | Facts stored permanently |
| **State Tracking** | Basic message history | Enhanced tracking of all intermediate results |
| **Efficiency** | May repeat searches | Can retrieve from memory first |

### Key Benefits of Advanced Implementation

1. **Long-Term Memory**
   - Facts learned from queries are stored permanently
   - Semantic search finds relevant past knowledge
   - Reduces redundant API calls and searches
   - Improves over time as it learns more

2. **Explicit State Machine**
   - Clear visualization of execution path
   - Each tool is a dedicated node/step
   - Deterministic flow (easier to test and debug)
   - Conditional branching based on evaluation
   - Better separation of concerns

3. **Enhanced State Tracking**
   - All intermediate results are stored in state
   - Easy to inspect what happened at each step
   - Better observability and debugging

### When to Use Which Approach

**Standard Agent (Part 02):**
- Simpler use cases
- Don't need persistent memory
- Flexible tool selection by LLM
- Quick prototyping

**Advanced Agent (Part 03):**
- Production systems requiring reliability
- Need to learn and remember facts over time
- Want clear, debuggable workflow
- Require audit trail of execution
- Performance optimization (reduce redundant calls)